In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [4]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1].values

In [5]:
from sklearn.preprocessing import StandardScaler

In [6]:
scaler = StandardScaler()

In [7]:
X = scaler.fit_transform(X)

In [8]:
X.shape

(768, 8)

In [9]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [46]:
import tensorflow
from tensorflow import keras
from keras.layers import Dense,Dropout
from keras import Sequential

In [11]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=8))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='Adam',loss="binary_crossentropy",metrics=['accuracy'])


c:\Users\hp\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [12]:
model.fit(X_train,y_train,batch_size=32,epochs=50,validation_data=(X_test,y_test))

Epoch 1/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.5674 - loss: 0.6875 - val_accuracy: 0.7208 - val_loss: 0.6289
Epoch 2/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6620 - loss: 0.6405 - val_accuracy: 0.7403 - val_loss: 0.5886
Epoch 3/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7283 - loss: 0.5835 - val_accuracy: 0.7597 - val_loss: 0.5602
Epoch 4/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7329 - loss: 0.5658 - val_accuracy: 0.7727 - val_loss: 0.5389
Epoch 5/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7070 - loss: 0.5718 - val_accuracy: 0.7792 - val_loss: 0.5220
Epoch 6/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7176 - loss: 0.5391 - val_accuracy: 0.7792 - val_loss: 0.5062
Epoch 7/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7316 - loss: 0.5207 - val_accuracy: 0.7727 - val_loss: 0.4944
Epoch 8/50
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7371 - loss: 0.5090 - val_accuracy: 0.7857 - val_loss

1. How to select appropriate optimizer
2. No. of nodes in a layer
3. how to select no. of layers
4. All in one model

In [13]:
import kerastuner as kt

C:\Users\hp\AppData\Local\Temp\ipykernel_4872\1654478174.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


1. How to select appropriate optimizer

In [14]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(32,activation='relu',input_dim=8))
    model.add(Dense(1,activation='sigmoid'))

    # optimizer=hp.Choice('optimizer',values = ['adam','sgmoid','rmsprop','adadelta']
    
    model.compile(optimizer=hp.Choice('optimizer',values = ['adam','sigmoid','rmsprop','adadelta']),loss="binary_crossentropy",metrics=['accuracy'])
    return model

In [15]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='best_optimizer')

Reloading Tuner from best_optimizer\untitled_project\tuner0.json


In [16]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))
tuner.results_summary()

Trial 4 Complete [00h 00m 02s]
val_accuracy: 0.7467532753944397

Best val_accuracy So Far: 0.7727272510528564
Total elapsed time: 00h 02m 52s
Results summary
Results in best_optimizer\untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
optimizer: rmsprop
Score: 0.7727272510528564

Trial 3 summary
Hyperparameters:
optimizer: adam
Score: 0.7467532753944397

Trial 0 summary
Hyperparameters:
optimizer: sigmoid
Traceback (most recent call last):
  File "c:\Users\hp\anaconda3\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\hp\anaconda3\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\hp\anaconda3\Lib\site-packages\kera

In [17]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [18]:
model = tuner.get_best_models(num_models=1)[0]
model.summary()

c:\Users\hp\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=5,validation_data=(X_test,y_test))

Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7708 - loss: 0.5176 - val_accuracy: 0.7792 - val_loss: 0.5087
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7514 - loss: 0.5262 - val_accuracy: 0.7987 - val_loss: 0.4974
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7806 - loss: 0.4933 - val_accuracy: 0.8052 - val_loss: 0.4881
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7591 - loss: 0.5018 - val_accuracy: 0.8052 - val_loss: 0.4817
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7761 - loss: 0.4814 - val_accuracy: 0.7987 - val_loss: 0.4772
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7510 - loss: 0.4938 - val_accuracy: 0.7987 - val_loss: 0.4735
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7936 - loss: 0.4794 - val_accuracy: 0.7922 - val_loss: 0.4707
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7931 - loss: 0.4633 - val_accuracy: 0.785

2. No. of neurons in a particular layer

In [20]:
def build_model(hp):
    model = Sequential()
    units = hp.Int('units',8,128,step=8)
    model.add(Dense(units, activation='relu', input_dim=8))
    model.add(Dense(1,activation='sigmoid'))

    model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])
    return model


In [21]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='neutrons_num')

In [22]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))
tuner.results_summary()

Trial 5 Complete [00h 00m 02s]
val_accuracy: 0.7922077775001526

Best val_accuracy So Far: 0.798701286315918
Total elapsed time: 00h 00m 09s
Results summary
Results in neutrons_num\untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 0 summary
Hyperparameters:
units: 96
Score: 0.798701286315918

Trial 2 summary
Hyperparameters:
units: 120
Score: 0.798701286315918

Trial 4 summary
Hyperparameters:
units: 88
Score: 0.7922077775001526

Trial 3 summary
Hyperparameters:
units: 24
Score: 0.7727272510528564

Trial 1 summary
Hyperparameters:
units: 72
Score: 0.7597402334213257


In [23]:
tuner.get_best_hyperparameters()[0].values

{'units': 96}

In [24]:
model = tuner.get_best_models(num_models=1)[0]

c:\Users\hp\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\hp\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [25]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=5,validation_data=(X_test,y_test))

Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7623 - loss: 0.5053 - val_accuracy: 0.7857 - val_loss: 0.4848
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7708 - loss: 0.4797 - val_accuracy: 0.7922 - val_loss: 0.4807
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7801 - loss: 0.4671 - val_accuracy: 0.7857 - val_loss: 0.4771
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7799 - loss: 0.4688 - val_accuracy: 0.7922 - val_loss: 0.4751
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7838 - loss: 0.4465 - val_accuracy: 0.7922 - val_loss: 0.4752
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7839 - loss: 0.4619 - val_accuracy: 0.7922 - val_loss: 0.4724
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7560 - loss: 0.4803 - val_accuracy: 0.7987 - val_loss: 0.4715
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7568 - loss: 0.4596 - val_accuracy: 0.785

3. How to select no. of layers

In [26]:
def build_model(hp):

    model = Sequential()
    model.add(Dense(96,activation='relu',input_dim=8))

    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        model.add(Dense(96,activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
    return model

In [27]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory='layers_num')

c:\Users\hp\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [28]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.8051947951316833

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 00m 14s


In [30]:
tuner.results_summary()

Results summary
Results in layers_num\untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
num_layers: 2
Score: 0.8181818127632141

Trial 0 summary
Hyperparameters:
num_layers: 3
Score: 0.8116883039474487

Trial 1 summary
Hyperparameters:
num_layers: 10
Score: 0.8116883039474487

Trial 4 summary
Hyperparameters:
num_layers: 7
Score: 0.8051947951316833

Trial 3 summary
Hyperparameters:
num_layers: 5
Score: 0.798701286315918


In [31]:
model = tuner.get_best_models(num_models=1)[0]

c:\Users\hp\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\hp\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [32]:
model.fit(X_train,y_train,epochs=100,initial_epoch=5,validation_data=(X_test,y_test))

Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.7825 - loss: 0.4644 - val_accuracy: 0.7987 - val_loss: 0.4794
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7782 - loss: 0.4608 - val_accuracy: 0.7792 - val_loss: 0.4911
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8020 - loss: 0.4311 - val_accuracy: 0.7922 - val_loss: 0.5042
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7652 - loss: 0.4761 - val_accuracy: 0.7987 - val_loss: 0.4901
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8018 - loss: 0.4086 - val_accuracy: 0.7792 - val_loss: 0.4993
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7870 - loss: 0.4199 - val_accuracy: 0.7922 - val_loss: 0.4810
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8236 - loss: 0.3969 - val_accuracy: 0.7857 - val_loss: 0.4885
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8235 - loss: 0.3990 - val_accuracy: 0.805

4. multiple things

In [47]:
def build_model(hp):
    model = Sequential()

    counter = 0

    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):

        if counter ==0:
            model.add(
                Dense(
                    units=hp.Int('units_' + str(i), min_value=8,max_value=128,step=8),
                    activation=hp.Choice('activation_' + str(i),values= ['relu', 'tanh','sigmoid']),
                    input_dim=8
                    )
                )
            model.add(Dropout(hp.Choice('dropout_' +str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        else:
            model.add(
                Dense(
                    units=hp.Int('units_' + str(i), min_value=8,max_value=128,step=8),
                    activation=hp.Choice('activation_' + str(i),values= ['relu', 'tanh','sigmoid']),
                    )
                )
            model.add(Dropout(hp.Choice('dropout_' +str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
        counter+=1

    model.add(Dense(1,activation='sigmoid'))
    model.compile(
        optimizer=hp.Choice('optimizer',values=['rmsprop','adam','sgd','nadam','adadelta']),
        loss='binary_crossentropy',
        metrics=['accuracy'])
    return model

In [49]:
tuner = kt.RandomSearch(build_model,
                        objective='val_accuracy',
                        max_trials=3,
                        directory='final')

c:\Users\hp\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [50]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 03s]
val_accuracy: 0.6428571343421936

Best val_accuracy So Far: 0.6428571343421936
Total elapsed time: 00h 00m 11s


In [43]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 1,
 'units_0': 24,
 'activation_0': 'tanh',
 'optimizer': 'rmsprop',
 'units_1': 56,
 'activation_1': 'relu',
 'units_2': 32,
 'activation_2': 'sigmoid',
 'units_3': 72,
 'activation_3': 'tanh',
 'units_4': 56,
 'activation_4': 'relu',
 'units_5': 48,
 'activation_5': 'relu',
 'units_6': 8,
 'activation_6': 'relu',
 'units_7': 112,
 'activation_7': 'relu',
 'units_8': 56,
 'activation_8': 'sigmoid'}

In [51]:
model = tuner.get_best_models(num_models=1)[0]

c:\Users\hp\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'nadam', because it has 2 variables whereas the saved optimizer has 39 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [52]:
model.fit(X_train,y_train,epochs=200,initial_epoch=5,validation_data=(X_test,y_test))

Epoch 6/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.5564 - loss: 0.6926 - val_accuracy: 0.6429 - val_loss: 0.6885
Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6246 - loss: 0.6891 - val_accuracy: 0.6429 - val_loss: 0.6856
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6503 - loss: 0.6846 - val_accuracy: 0.6429 - val_loss: 0.6826
Epoch 9/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6250 - loss: 0.6833 - val_accuracy: 0.6429 - val_loss: 0.6798
Epoch 10/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6527 - loss: 0.6777 - val_accuracy: 0.6429 - val_loss: 0.6769
Epoch 11/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6214 - loss: 0.6782 - val_accuracy: 0.6429 - val_loss: 0.6741
Epoch 12/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6424 - loss: 0.6733 - val_accuracy: 0.6429 - val_loss: 0.6712
Epoch 13/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6506 - loss: 0.6674 - val_accuracy: 0.642